In [1]:
!pip install numpy pandas matplotlib scikit-learn pyts torch tensorflow

In [2]:
# Create directories
!mkdir -p data/Indoor_User_Movement_Prediction
!mkdir -p data/Daily_and_Sports_Activity

# Download zip files from UCI (replace these URLs with the actual direct download links)
!wget -O data/Indoor_User_Movement_Prediction.zip "https://archive.ics.uci.edu/static/public/348/indoor+user+movement+prediction+from+rss+data.zip"
!wget -O data/Daily_and_Sports_Activity.zip "https://archive.ics.uci.edu/static/public/256/daily+and+sports+activities.zip"

# Unzip files into the correct folders
!unzip -q data/Indoor_User_Movement_Prediction.zip -d data/Indoor_User_Movement_Prediction
!unzip -q data/Daily_and_Sports_Activity.zip -d data/Daily_and_Sports_Activity

--2025-05-06 11:34:14--  https://archive.ics.uci.edu/static/public/348/indoor+user+movement+prediction+from+rss+data.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data/Indoor_User_Movement_Prediction.zip’

data/Indoor_User_Mo     [   <=>              ] 209.99K   346KB/s    in 0.6s    

2025-05-06 11:34:15 (346 KB/s) - ‘data/Indoor_User_Movement_Prediction.zip’ saved [215028]

--2025-05-06 11:34:15--  https://archive.ics.uci.edu/static/public/256/daily+and+sports+activities.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data/Daily_and_Sports_Activity.zip’

data/Daily_and_Spor     [    <=>             

In [3]:
import sys
import warnings
import logging

# --- Logging setup for both file and console ---
logger = logging.getLogger('dual_logger')
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# File handler (will not log warnings)
class NoWarningFileHandler(logging.FileHandler):
    def emit(self, record):
        if record.levelno != logging.WARNING:
            super().emit(record)

file_handler = NoWarningFileHandler('results_output.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)

# Console handler (shows everything)
console_handler = logging.StreamHandler(sys.__stdout__)
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(console_handler)

# --- Show each warning only once ---
warnings.simplefilter('once')

# --- Redirect print statements to logger.info ---
class LoggerWriter:
    def __init__(self, level_func):
        self.level_func = level_func
    def write(self, message):
        message = message.rstrip()
        if message:
            self.level_func(message)
    def flush(self):
        pass

sys.stdout = LoggerWriter(logger.info)
sys.stderr = LoggerWriter(logger.error)

# --- Suppress warnings from being logged to file ---
def custom_showwarning(message, category, filename, lineno, file=None, line=None):
    # Only print warnings to console, not to file
    sys.__stderr__.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = custom_showwarning

In [4]:
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, Conv1D, BatchNormalization, Activation, GlobalAveragePooling1D, Add, Flatten, PReLU, MaxPooling1D, Lambda, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GroupNormalization

def build_mlp(input_shape, nb_classes):
    x = Input(shape=input_shape)
    x_nb = Dense(64, activation='relu')(x)
    x_nb = Dropout(0.2)(x_nb)
    out = Dense(nb_classes, activation='softmax')(x_nb)
    return x, out

def build_lstm(input_shape, nb_classes):
    x = Input(shape=input_shape)
    x_nb = LSTM(64)(x)
    x_nb = Dropout(0.2)(x_nb)
    out = Dense(nb_classes, activation='softmax')(x_nb)
    return x, out

def build_fcn(input_shape, nb_classes):
    x = Input(shape=input_shape)
    conv_x = BatchNormalization()(x)
    conv_x = Conv1D(128, kernel_size=8, padding='same')(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Dropout(0.2)(conv_x)
    full = GlobalAveragePooling1D()(conv_x)
    out = Dense(nb_classes, activation='softmax')(full)
    return x, out

def build_resnet(input_shape, n_feature_maps, nb_classes):
    x = Input(shape=input_shape)
    conv_x = BatchNormalization()(x)
    conv_x = Conv1D(n_feature_maps, kernel_size=8, padding='same')(conv_x)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_y = Conv1D(n_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
    conv_z = Conv1D(n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = BatchNormalization()(conv_z)
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = Conv1D(n_feature_maps, kernel_size=1, padding='same')(x)
        shortcut_y = BatchNormalization()(shortcut_y)
    else:
        shortcut_y = BatchNormalization()(x)
    y = Add()([shortcut_y, conv_z])
    y = Activation('relu')(y)
    x1 = y
    conv_x = Conv1D(n_feature_maps * 2, kernel_size=8, padding='same')(x1)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_y = Conv1D(n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
    conv_z = Conv1D(n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = BatchNormalization()(conv_z)
    is_expand_channels = not (input_shape[-1] == n_feature_maps * 2)
    if is_expand_channels:
        shortcut_y = Conv1D(n_feature_maps * 2, kernel_size=1, padding='same')(x1)
        shortcut_y = BatchNormalization()(shortcut_y)
    else:
        shortcut_y = BatchNormalization()(x1)
    y = Add()([shortcut_y, conv_z])
    y = Activation('relu')(y)
    x1 = y
    conv_x = Conv1D(n_feature_maps * 2, kernel_size=8, padding='same')(x1)
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_y = Conv1D(n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
    conv_z = Conv1D(n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = BatchNormalization()(conv_z)
    is_expand_channels = not (input_shape[-1] == n_feature_maps * 2)
    if is_expand_channels:
        shortcut_y = Conv1D(n_feature_maps * 2, kernel_size=1, padding='same')(x1)
        shortcut_y = BatchNormalization()(shortcut_y)
    else:
        shortcut_y = BatchNormalization()(x1)
    y = Add()([shortcut_y, conv_z])
    y = Activation('relu')(y)
    full = GlobalAveragePooling1D()(y)
    out = Dense(nb_classes, activation='softmax')(full)
    return x, out

def build_encoder(input_shape, nb_classes):
    x = Input(input_shape)
    conv1 = Conv1D(filters=128, kernel_size=5, strides=1, padding='same')(x)
    conv1 = GroupNormalization(groups=-1, axis=-1)(conv1)
    conv1 = PReLU(shared_axes=[1])(conv1)
    conv1 = Dropout(rate=0.2)(conv1)
    conv1 = MaxPooling1D(pool_size=2)(conv1)
    conv2 = Conv1D(filters=256, kernel_size=11, strides=1, padding='same')(conv1)
    conv2 = GroupNormalization(groups=-1, axis=-1)(conv2)
    conv2 = PReLU(shared_axes=[1])(conv2)
    conv2 = Dropout(rate=0.2)(conv2)
    conv2 = MaxPooling1D(pool_size=2)(conv2)
    conv3 = Conv1D(filters=512, kernel_size=21, strides=1, padding='same')(conv2)
    conv3 = GroupNormalization(groups=-1, axis=-1)(conv3)
    conv3 = PReLU(shared_axes=[1])(conv3)
    conv3 = Dropout(rate=0.2)(conv3)
    attention_data = Lambda(lambda x: x[:, :, :256])(conv3)
    attention_softmax = Lambda(lambda x: x[:, :, 256:])(conv3)
    attention_softmax = keras.layers.Softmax()(attention_softmax)
    multiply_layer = Multiply()([attention_softmax, attention_data])
    dense_layer = Dense(units=256, activation='sigmoid')(multiply_layer)
    dense_layer = GroupNormalization(groups=-1, axis=-1)(dense_layer)
    flatten_layer = Flatten()(dense_layer)
    out = Dense(units=nb_classes, activation='softmax')(flatten_layer)
    return x, out


In [5]:
from pyts.metrics import dtw, boss
from scipy.stats import wasserstein_distance
import numpy as np

def boss_metric(x, y):
    return boss(x, y)

def dtw_classic(x, y):
    return dtw(x, y, method='classic')

def dtw_sakoechiba(x, y, window_size=0.5):
    return dtw(x, y, method='sakoechiba', options={'window_size': window_size})

def dtw_itakura(x, y, max_slope=1.5):
    return dtw(x, y, method='itakura', options={'max_slope': max_slope})

def dtw_multiscale(x, y, resolution=2):
    return dtw(x, y, method='multiscale', options={'resolution': resolution})

def dtw_fast(x, y, radius=1):
    return dtw(x, y, method='fast', options={'radius': radius})

def wasserstein_metric(x, y):
    return wasserstein_distance(x, y)

def cal_similarity(view1, view2, metric='boss'):
    similarity_list = []
    for i in range(view1.shape[0]):
        x = np.squeeze(view1[i])
        y = np.squeeze(view2[i])
        if metric == 'boss':
            similarity_list.append(boss(x, y))
        elif metric == 'dtw_classic':
            similarity_list.append(dtw_classic(x, y))
        elif metric == 'dtw_sakoechiba':
            similarity_list.append(dtw_sakoechiba(x, y, window_size=0.5))
        elif metric == 'dtw_itakura':
            similarity_list.append(dtw_itakura(x, y, max_slope=1.5))
        elif metric == 'dtw_multiscale':
            similarity_list.append(dtw_multiscale(x, y, resolution=2))
        elif metric == 'dtw_fast':
            similarity_list.append(dtw_fast(x, y, radius=1))
        elif metric == 'wasserstein':
            similarity_list.append(wasserstein_metric(x, y))
        else:
            print('other metric not implement yet.')
    return np.array(similarity_list)


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.neighbors import KernelDensity
from tensorflow.keras.metrics import Precision, Recall, AUC, TopKCategoricalAccuracy
import os

# --- TPU strategy setup ---
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("Running on TPU!")
except Exception as e:
    strategy = tf.distribute.get_strategy()  # Default strategy (CPU/GPU)
    print("TPU not found, running on CPU/GPU.")

def print_metrics(history):
    for metric_name, values in history.history.items():
        print(f"{metric_name}: {values[-1]:.4f}")

def prepare_views(data_train, data_test):
    views = {}
    for i in range(data_train.shape[2]):
        views[f'view{i}_train'] = data_train[:,:,i].reshape(data_train.shape[0], data_train.shape[1], 1)
        views[f'view{i}_test'] = data_test[:,:,i].reshape(data_test.shape[0], data_test.shape[1], 1)
    return views

def calculate_similarities(views, source_idx, metric):
    similarities = {}
    n_views = len([k for k in views if '_train' in k])
    for i in range(n_views):
        if i != source_idx:
            similarities[f'similarity{source_idx}{i}'] = cal_similarity(
                views[f'view{source_idx}_train'],
                views[f'view{i}_train'],
                metric
            )
    return similarities

def calculate_weights(similarities, source_idx):
    kde_models = {}
    weights = {}
    weight_all = 0
    target_indices = [int(k[len(f'similarity{source_idx}') :]) for k in similarities.keys()]
    for i in target_indices:
        kde_key = f'kde{source_idx}{i}'
        sim_key = f'similarity{source_idx}{i}'
        sim_data = similarities[sim_key]
        if sim_data.shape[0] == 0:
            print(f"Warning: No samples for {sim_key}, skipping KDE fit.")
            continue
        kde_models[kde_key] = KernelDensity(kernel='gaussian', bandwidth=7.8).fit(
            sim_data.reshape(sim_data.flatten().shape[0], 1)
        )
        weight_key = f'weight{source_idx}{i}'
        weights[weight_key] = np.mean(kde_models[kde_key].sample(10, random_state=0), axis=0)[0]
        weight_all += weights[weight_key]
    return kde_models, weights, weight_all

def train_model(model_type, views, source_idx, target_train, target_test, nb_classes, transfer_type='no_transfer', weights=None, weight_all=None):
    with strategy.scope():
        if model_type == 'fcn':
            x, y = build_fcn(views[f'view0_train'].shape[1:], nb_classes)
        else: # lstm
            x, y = build_lstm(views[f'view0_train'].shape[1:], nb_classes)
        model = keras.models.Model(inputs=x, outputs=y)
        adam = Adam(learning_rate=0.005)
        model.compile(
            loss='categorical_crossentropy',
            optimizer=adam,
            metrics=['accuracy', Precision(), Recall(), AUC(), TopKCategoricalAccuracy(k=5)],
            steps_per_execution=50  # improves TPU throughput[1]
        )

    n_views = len([k for k in views if '_train' in k])
    target_indices = [i for i in range(n_views) if i != source_idx]
    if transfer_type in ['naive_transfer', 'weighted_transfer']:
        for i in target_indices:
            if transfer_type == 'naive_transfer':
                epochs = 30
            else: # weighted_transfer
                weight_key = f'weight{source_idx}{i}'
                if weights is not None and weight_key in weights and weight_all:
                    epochs = int(30 * 7 * weights[weight_key] / weight_all) + 1
                else:
                    print(f"Warning: {weight_key} not found in weights. Skipping this view.")
                    continue
            model.fit(
                views[f'view{i}_train'],
                to_categorical(target_train, nb_classes),
                epochs=epochs,
                batch_size=128,
                validation_data=(views[f'view{i}_test'], to_categorical(target_test, nb_classes)),
                verbose=0
            )
    epochs = 40 if transfer_type in ['no_transfer', 'weighted_transfer'] else 30
    history = model.fit(
        views[f'view{source_idx}_train'],
        to_categorical(target_train, nb_classes),
        epochs=epochs,
        batch_size=128,
        validation_data=(views[f'view{source_idx}_test'], to_categorical(target_test, nb_classes)),
        verbose=0
    )
    return history

def run_training(data_train, data_test, target_train, target_test, source_idx=0, model_types=['fcn', 'lstm'], nb_classes=2):
    metrics = ['boss', 'dtw_classic', 'dtw_sakoechiba', 'dtw_itakura', 'dtw_multiscale', 'wasserstein']
    results = {model_type: {} for model_type in model_types}
    views = prepare_views(data_train, data_test)
    n_views = len([k for k in views if '_train' in k])
    for metric in metrics:
        mode_name = f'{metric}+kde'
        print(f"\n===== Processing {mode_name} =====")
        similarities = calculate_similarities(views, source_idx, metric)
        kde_models, weights, weight_all = calculate_weights(similarities, source_idx)
        for model_type in model_types:
            results[model_type][mode_name] = {}
            print(f'*** {model_type.upper()} - {mode_name} - No transfer learning ***')
            history = train_model(model_type, views, source_idx, target_train, target_test, nb_classes)
            results[model_type][mode_name]['No_transfer'] = {
                metric: history.history[metric] for metric in history.history if metric.startswith('val_')
            }
            print(f"Final accuracy: {history.history['val_accuracy'][-1]}")
            print_metrics(history)
            print(f'*** {model_type.upper()} - {mode_name} - Naive transfer learning ***')
            history = train_model(model_type, views, source_idx, target_train, target_test, nb_classes, transfer_type='naive_transfer')
            results[model_type][mode_name]['Naive_Transfer'] = {
                metric: history.history[metric] for metric in history.history if metric.startswith('val_')
            }
            print(f"Final accuracy: {history.history['val_accuracy'][-1]}")
            print_metrics(history)
            print(f'*** {model_type.upper()} - {mode_name} - Weighted transfer learning ***')
            history = train_model(model_type, views, source_idx, target_train, target_test, nb_classes, transfer_type='weighted_transfer', weights=weights, weight_all=weight_all)
            results[model_type][mode_name]['Weighted_Transfer'] = {
                metric: history.history[metric] for metric in history.history if metric.startswith('val_')
            }
            print(f"Final accuracy: {history.history['val_accuracy'][-1]}")
            print_metrics(history)
    return results


In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import os

def load_sensor_data(data_root):
    segments, labels, metadata = [], [], []
    for activity in sorted(os.listdir(data_root)):
        activity_path = os.path.join(data_root, activity)
        if not os.path.isdir(activity_path): continue
        for person in sorted(os.listdir(activity_path)):
            person_path = os.path.join(activity_path, person)
            if not os.path.isdir(person_path): continue
            for session in sorted(os.listdir(person_path)):
                if session.endswith('.txt'):
                    path = os.path.join(person_path, session)
                    raw = np.loadtxt(path, delimiter=",")
                    segments.append(raw)  # shape (125, 45)
                    labels.append(activity)
                    metadata.append((activity, person, session))
    return segments, labels, metadata

def scale_segments(segment_list):
    return [MinMaxScaler(feature_range=(-1, 1)).fit_transform(seg) for seg in segment_list]

def segment_windows(segments, labels, metadata):
    X, y, meta_out = [], [], []
    for segment, label, meta in zip(segments, labels, metadata):
        X.append(segment.T)  # (45, 125)
        y.append(label)
        meta_out.append(meta)
    return np.stack(X), np.array(y), meta_out

def split_body_domains(X):
    return {
        'torso': X[:, 0:9, :],
        'right_arm': X[:, 9:18, :],
        'left_arm': X[:, 18:27, :],
        'right_leg': X[:, 27:36, :],
        'left_leg': X[:, 36:45, :]
    }

# Load, scale, and prepare data
data_root = "data/Daily_and_Sports_Activity/data/"
segments, labels, metadata = load_sensor_data(data_root)
segments = scale_segments(segments)
X, y, meta = segment_windows(segments, labels, metadata)
le = LabelEncoder()
y_int = le.fit_transform(y)
domains = split_body_domains(X)
view = domains['torso']  # (samples, 9, 125)
X_view = np.transpose(view, (0, 2, 1))  # (samples, 125, 9)
X_train, X_test, y_train, y_test = train_test_split(X_view, y_int, test_size=0.2, random_state=42, stratify=y_int)
data_train, data_test, targets_train, targets_test = X_train, X_test, y_train, y_test
nb_classes = len(np.unique(targets_train))

# --- Run training loop ---
results = run_training(
    data_train, data_test, targets_train, targets_test,
    source_idx=0, model_types=['fcn', 'lstm'], nb_classes=nb_classes
)
print("\n=== Final Validation Accuracies ===")
for model_type in results:
    print(f"\nModel: {model_type.upper()}")
    for mode_name in results[model_type]:
        print(f" {mode_name}:")
        for strategy in results[model_type][mode_name]:
            metrics_dict = results[model_type][mode_name][strategy]
            if metrics_dict:
                for metric in sorted(metrics_dict):
                  curve = metrics_dict[metric]
                  print(f"  {strategy} - {metric}: {curve[-1]:.4f}")
            else:
                print(f"  {strategy}: No data")